In [10]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0";


import keras
from keras.applications.vgg16 import VGG16
from keras.applications.densenet import DenseNet121

In [11]:
import numpy as np
x_train=np.load("x_train.npy")
x_test=np.load("x_test.npy")
y_train=np.load("y_train.npy")
y_test=np.load("y_test.npy")
print(y_test.shape)

(19850, 397)


In [12]:
from keras.applications.densenet import preprocess_input

x_test = preprocess_input(x_test)
x_train = preprocess_input(x_train)

In [13]:
model = DenseNet121(weights='imagenet', include_top=False, classes=397, input_shape=(224,224,3))
print(model.summary())

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
zero_padding2d_1 (ZeroPadding2D (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1/conv (Conv2D)             (None, 112, 112, 64) 9408        zero_padding2d_1[0][0]           
__________________________________________________________________________________________________
conv1/bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1/conv[0][0]                 
__________________________________________________________________________________________________
conv1/relu

In [14]:
from keras.layers import Flatten, Dense, Dropout
from keras.models import Model
from keras.layers.pooling import GlobalAveragePooling2D

# x = model.get_layer('block5_pool').output
# x = Flatten(name='flatten')(x)
# x = Dense(512, activation='relu', name='fc1')(x)
# x = Dense(4096, activation='relu', name='fc2')(x)
# x = Dropout(0.3)(x)
# x = Dense(397, activation='softmax', name='predictions')(x)
x = model.get_layer('relu').output
x = GlobalAveragePooling2D(name='pool')(x)
x = Dense(397, activation='softmax', name='fc1')(x)

model_updated = Model(inputs=model.input, outputs=x)

In [18]:
# def learning_rate_schedule(epoch):
#     if epoch <= 10:
#         return 1e-4 # 0.00001
#     elif epoch <= 20:
#         return 1e-5
#     elif epoch <= 30:
#         return 1e-6
#     else:
#         return 1e-7
#     return LR
def learning_rate_schedule(epoch):
    if epoch <= 5:
        return 1e-5 # 0.000001
    elif epoch <= 10:
        return 1e-6
    elif epoch <= 15:
        return 1e-7
    else:
        return 1e-8
    return LR

In [7]:
model_updated.save_weights('model_initial.h5')

In [19]:
from keras import optimizers
from keras.callbacks import EarlyStopping

model_updated.load_weights('model1.h5')
training_runs = []
# for i in range(5):
#     if i!=0:
# #         model_updated.load_weights('model_initial.h5')
# #     else:
#         model_updated.load_weights('model1.h5')
model_updated.compile(loss='categorical_crossentropy', optimizer=optimizers.adam(lr=0.0001), metrics=['accuracy'])
keras.callbacks.LearningRateScheduler(learning_rate_schedule)
history = model_updated.fit(x_train, y_train, batch_size=64, shuffle=True, epochs=20, validation_data=(x_test, y_test))
training_runs.append(history)
model_updated.get_weights()
model_updated.save_weights('model1.h5')

Train on 19850 samples, validate on 19850 samples
Epoch 1/20
19850/19850 [==============================] - 235s 12ms/step - loss: 0.0641 - acc: 0.9950 - val_loss: 1.9916 - val_acc: 0.5144
Epoch 2/20
19850/19850 [==============================] - 199s 10ms/step - loss: 0.0494 - acc: 0.9961 - val_loss: 1.8887 - val_acc: 0.5418
Epoch 3/20
19850/19850 [==============================] - 199s 10ms/step - loss: 0.0321 - acc: 0.9980 - val_loss: 1.9439 - val_acc: 0.5339
Epoch 4/20
19850/19850 [==============================] - 199s 10ms/step - loss: 0.0159 - acc: 0.9992 - val_loss: 1.9404 - val_acc: 0.5378
Epoch 5/20
19850/19850 [==============================] - 199s 10ms/step - loss: 0.0918 - acc: 0.9865 - val_loss: 2.0729 - val_acc: 0.5145
Epoch 6/20
19850/19850 [==============================] - 199s 10ms/step - loss: 0.0416 - acc: 0.9956 - val_loss: 1.9773 - val_acc: 0.5345
Epoch 7/20
19850/19850 [==============================] - 199s 10ms/step - loss: 0.0122 - acc: 0.9994 - val_loss: 1.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.subplot(2, 3, 1)
plt.plot(training_runs[0].history['acc'])
plt.legend(['train'], loc='lower right')

plt.subplot(2, 3, 2)
plt.plot(training_runs[1].history['acc'])
plt.legend(['train'], loc='lower right')

plt.subplot(2, 3, 3)
plt.plot(training_runs[2].history['acc'])
plt.legend(['train'], loc='lower right')

plt.subplot(2, 3, 4)
plt.plot(training_runs[0].history['val_acc'])
plt.legend(['test'], loc='lower right')

plt.subplot(2, 3, 5)
plt.plot(training_runs[1].history['val_acc'])
plt.legend(['test'], loc='lower right')

plt.subplot(2, 3, 6)
plt.plot(training_runs[2].history['val_acc'])
plt.legend(['test'], loc='lower right')

plt.tight_layout() 

plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
print("Average training accuracy: {}".format(np.mean([training_runs[0].history['acc'][-1], 
                                                     training_runs[1].history['acc'][-1], training_runs[2].history['acc'][-1]])))
print("Average testing accuracy: {}".format(np.mean([training_runs[0].history['val_acc'][-1], 
                                                     training_runs[1].history['val_acc'][-1], training_runs[2].history['val_acc'][-1]])))